In [ ]:
import tensorflow as tf

In [ ]:
path = 'fruits360_filtered/'

In [ ]:
train_dir = path + 'Training'
validation_dir = path + 'Test'

In [ ]:
total_train = 11398
total_val = 4752

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

In [ ]:
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
channel = 3

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           target_size=(img_height, img_width))

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(img_height, img_width))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
np.random.seed(8)
tf.random.set_seed(8)

In [ ]:
from tensorflow.keras.applications import NASNetMobile

In [ ]:
base_model = NASNetMobile(include_top=False, input_shape=(img_height, img_width, channel), weights='imagenet')

In [ ]:
for layer in base_model.layers[:700]:
  layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.Dense(120, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [ ]:
model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)